<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [ ]:
# !pip install emoji
# !pip install nltk

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5a92f91dcf134fc24011694b0e97f509edb85899e2f6e4aeb666514621a3bb94
  Stored in directory: c:\users\trist\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


### Sort raw data by time

In [ ]:
@timer('ms')

def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()

    with open(json_file, 'r+') as file:
        content = file.read()
        content=json.loads(content)
        df = pd.DataFrame(content)
        df = df.T

    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df.iloc[:, 6]), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date

        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].replace('\n', '')
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')

In [ ]:
raw_files = ['train.data.txt', 'dev.data.txt']
json_files = ['./filtered data/train_reply.json', './filtered data/dev_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

In [ ]:
with open(raw_files[0]) as file:
    train = file.readlines()
with open('train.data_sorted.txt') as file:
    s = file.readlines()
assert len(train) == len(s)

## Text pre-processing

In [2]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd
from utils import timer
from datetime import datetime
# from nltk.corpus import wordnet

In [3]:
nltk.download('wordnet')
stemmer = nltk.stem.porter.PorterStemmer()
stopword = stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yuhandan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:



@timer('ms')
# def replace_duplicate_letter(word):
#     repeat_reg = re.compile(r'(\w*)(\w)\2(\w*)')
#     repl = r'\1\2\3'
#     if wordnet.synsets(word):  # 判断当前字符串是否是单词
#         return word
#     repl_word = repeat_reg.sub(repl, word)
#     if repl_word != word:
#         return replace_duplicate_letter(repl_word)
#     else:
#         return repl_word
  
def clean_tweet(content):
    from time import strftime
    # def compute_num_month(content):
    #     month_num = 0
    #     month = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "octorber", "november", "december", 
    #             "jan.", "feb.", "mar.", "apr.", "may.", "jun.", "jul.", "aug.", "sept.", "oct.", "nov.", "dec."]
    #     for i in content:
    #         if i in month:
    #             month_num += 1
    #     return month_num
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won't", "will not", content)
    content = re.sub(r"can't", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n't", " not", content)
    content = re.sub(r"'re", " are", content)
    content = re.sub(r"'s", " is", content)
    content = re.sub(r"'d", " would", content)
    content = re.sub(r"'ll", " will", content)
    content = re.sub(r"'t", " not", content)
    content = re.sub(r"'ve", " have", content)
    content = re.sub(r"'m", " am", content)
    content = re.sub(r".”", " ", content)
    
    # get the number of month be mentioned
    # month_num = compute_num_month(content)

    # get the number of url
    mentioned_url_num = len(re.findall(r'https?://[^ ]+', content))
    mentioned_url_num += len(re.findall(r'www.[^ ]+', content))
    # get the number of twitter ID be mentioned
    id_num = len(re.findall(r'@[A-Za-z0-9_]+', content))
    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    
    content = re.sub(r'[:_!+“\-=——,$%^\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation, except .
    
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    
    # remove stop words 
    # TODO: and keep only english letters
    
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token.strip()) for token in content]
    
    return ' '.join(content), mentioned_url_num, id_num 
    #, month_num

In [5]:
@timer('ms')
def json2df(json_file):
    """
    json_file: 'train_reply.json'
    """
    with open(json_file,'r+') as file:
        content = file.read()
    content = json.loads(content)
    df = pd.DataFrame(content)
    df = df.T
    return df
# df = json2df('train_reply.json')
def clean_data(data_type):
  """
  Args: 
    data_type: 'dev', 'train'
  Returns:
    source_df, reply_df
  """
  source_df = json2df(f'./data/full data/{data_type}_source.json')
  source_df['temp'] = source_df['text'].apply(lambda x: clean_tweet(x))
  source_df['text'] = source_df['temp'].apply(lambda x: x[0])
  source_df['mentioned_url_num'] = source_df['temp'].apply(lambda x: x[1])
  source_df['id_num'] = source_df['temp'].apply(lambda x: x[2])
  source_df = source_df.drop(columns='temp')
  source_df['tweet_id'] = source_df.index

  reply_df = json2df(f'./data/full data/{data_type}_reply.json')
  reply_df['temp'] = reply_df['text'].apply(lambda x: clean_tweet(x))
  reply_df['text'] = reply_df['temp'].apply(lambda x: x[0])
  reply_df['mentioned_url_num'] = reply_df['temp'].apply(lambda x: x[1])
  reply_df['id_num'] = reply_df['temp'].apply(lambda x: x[2])
  reply_df = reply_df.drop(columns='temp')
  reply_df['tweet_id'] = reply_df.index
  
  return source_df, reply_df
# train_source_df, train_reply_df = clean_data('train')

# dev_source_df, dev_reply_df = clean_data('dev')


## Concat label

In [6]:
import pandas as pd
def concat_label(data_type, source_feature_df):
    """Concat labels on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'label'])
    with open(f'./data/original_data/{data_type}_source.txt', 'r') as f:
        ids = f.readlines()
    with open(f'./data/original_data/{data_type}.label.txt', 'r') as f:
        labels = f.readlines()
    df['tweet_id'] = [id.strip() for id in ids]
    df['label'] = [label.strip() for label in labels]
    df_labels = pd.merge(source_feature_df, df, on='tweet_id', how='left')
    df_labels['label'] = df_labels['label'].apply(lambda x: 0 if x == 'nonrumour' else 1)
    return df_labels

# dev_source_df = concat_label('dev', dev_source_df)
# train_source_df = concat_label('train', train_source_df)


## Concat Sorted reply

In [7]:
def concat_reply(data_type, source_df):
    """concat replies on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'reply'])
    with open(f'./data/original_data/{data_type}.data_sorted.txt', 'r') as f:
        content = f.readlines()
    df['tweet_id'] = [c.split(',')[0].strip() for c in content]
    df['reply'] = [','.join([i.strip() for i in c.split(',')[1:]]) for c in content]
    source_df = pd.merge(source_df, df, on='tweet_id', how='left')
    return source_df

# dev_source_df = concat_reply('dev', dev_source_df)
# train_source_df = concat_reply('train', train_source_df)


## check whether the created date is on weekday

In [8]:
def check_weekday(df):
    """
    df: source_df or reply_df
    """
    df['isoweekday'] = df['created_at'].apply(lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d').isoweekday())
    df['isweekday'] = df['isoweekday'].apply(lambda x: 1 <= x <= 5)
    df = df.drop(columns='isoweekday')
    return df

## Concat user info

In [9]:
def concat_user_info(data_type, source_df):
    """concat user info on source tweets
    data_type: 'dev', 'train'
    """
    df = json2df(f'./data/full data/{data_type}_source_userinfo.json')
    df['user_id'] = df.index
    source_df = pd.merge(source_df, df, on='user_id', how='left')
    return source_df



# Processing

In [10]:
def processing(data_type):
    source_df, reply_df = clean_data(data_type)
    source_df = concat_label(data_type, source_df)
    source_df = concat_reply(data_type, source_df)
    source_df = check_weekday(source_df)
    source_df = concat_user_info(data_type, source_df)
    reply_df = check_weekday(reply_df)
    return source_df, reply_df

In [11]:
train_source_df, train_reply_df = processing('train')
dev_source_df, dev_reply_df = processing('dev')

Call json2df in 71.7310905456543 ms
Call clean_tweet in 7.306098937988281 ms
Call clean_tweet in 0.20813941955566406 ms
Call clean_tweet in 1.2710094451904297 ms
Call clean_tweet in 0.31495094299316406 ms
Call clean_tweet in 0.49042701721191406 ms
Call clean_tweet in 0.21409988403320312 ms
Call clean_tweet in 0.2980232238769531 ms
Call clean_tweet in 0.6420612335205078 ms
Call clean_tweet in 0.3497600555419922 ms
Call clean_tweet in 0.38504600524902344 ms
Call clean_tweet in 0.3001689910888672 ms
Call clean_tweet in 0.20694732666015625 ms
Call clean_tweet in 0.13589859008789062 ms
Call clean_tweet in 0.2789497375488281 ms
Call clean_tweet in 0.95367431640625 ms
Call clean_tweet in 0.5550384521484375 ms
Call clean_tweet in 1.1219978332519531 ms
Call clean_tweet in 1.2280941009521484 ms
Call clean_tweet in 0.4017353057861328 ms
Call clean_tweet in 0.3039836883544922 ms
Call clean_tweet in 1.0478496551513672 ms
Call clean_tweet in 0.2512931823730469 ms
Call clean_tweet in 0.23484230041503

In [21]:
train_source_df.to_csv('./data/filtered_data/train_source_df.csv')
train_reply_df.to_csv('./data/filtered_data/train_reply_df.csv')
dev_source_df.to_csv('./data/filtered_data/dev_source_df.csv')
dev_reply_df.to_csv('./data/filtered_data/dev_reply_df.csv')
